<a href="https://colab.research.google.com/github/ShreyasJothish/blognotebooks/blob/master/DS1_Predictive_Modeling_Challenge_Parameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the data from Kaggle
!pip install kaggle

# Upgrade the version of Seaborn
!pip install -U seaborn

# Install category_encoders
!pip install category_encoders

In [0]:
# Mount the drive to download the data from Kaggle
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive

!kaggle competitions download -c ds1-predictive-modeling-challenge

In [0]:
# Extract the csv files
!unzip train_features.csv.zip 
!unzip train_labels.csv.zip 
!unzip test_features.csv.zip

In [0]:
# Generic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# Loading the independent features as X and
# dependent variable as y
nan_values_list = ['Not Known', 'Unknown', 'None', 'Not known', 'not known', 
                   '-', 'unknown', 'Unknown Installer', '##', 'none']

train_features_df = pd.read_csv('train_features.csv', na_values=nan_values_list)
train_labels_df = pd.read_csv('train_labels.csv')


In [0]:
def atleast(row, value_count_series, count=5):
  # Identify items who have funded atleast 5 pumps
  if str(row) == "nan":
    return np.nan
  
  value_count = value_count_series.get(row)
  
  if value_count < count:
    return 0
  else:
    return 1

def character_grouping(row):
  # Reduce the dimension based on 1st character else return *
  if str(row) == "nan":
    return np.nan
  
  if row[0].isalpha():
    return row[0].lower()
  else:
    return "*"
  
def classify_lga(row):
  # Classify lga into Rural, Urban and others
  if str(row) == "nan":
    return np.nan
  
  if row.lower().find('rural'):
    return "rural"
  elif row.lower().find('urban'):
    return "urban"
  else:
    return "other"
  
def prefix_grouping(row, prefix_count=3):
  # Reduce the dimension based on 1st character else return *
  if str(row) == "nan":
    return np.nan
  
  if prefix_count > len(row):
    return "#"
  
  if row[0:prefix_count].isalpha():
    return row[0:prefix_count].lower()
  else:
    return "*"
  
def map_ward_construction_year(input_df):
  # Map ward to construction year
  
  # Here train_features_df shall be used as reference for
  # both trainging and test data set.
  df = input_df.copy()
  
  ward_construction_year_dict = {}
  ward_list = df['ward'].unique()
 
  # top ward's construction year shall be used incase there is no
  # matching construction year for individual ward.  
  top_ward = df['ward'].describe().top
  top_ward_construction_year =  \
    int(df[df['ward'] == top_ward]['construction_year'].median())

  for ward in ward_list:
    ward_construction_year = \
      int(df[df['ward'] == ward]['construction_year'].median())
    
    if ward not in ward_construction_year_dict:\
      
      if ward_construction_year == 0:
        ward_construction_year_dict[ward] = top_ward_construction_year
      else:
        ward_construction_year_dict[ward] = ward_construction_year
  
  return ward_construction_year_dict


def compute_construction_year(row, ward_construction_year_dict, 
                              top_ward_construction_year):
  # compute the consturction year if it is 0  
  ward = row['ward']
  construction_year = row['construction_year']  
  
  if construction_year == 0:
    if ward in ward_construction_year_dict:
      return ward_construction_year_dict[ward]
    else:
      return top_ward_construction_year
  else:
    return construction_year


def compute_age(row):
  # compute the consturction age
  date_recorded = row['date_recorded']
  year_recorded = int(date_recorded.split('-')[0])
  
  construction_year = row['construction_year']
  
  return (year_recorded - construction_year)


def compute_year_recorded(row):
  # split year from date_recorded
  return int(row.split('-')[0])

def compute_month_recorded(row):
  # split year from date_recorded
  return int(row.split('-')[1])

In [0]:
def feature_engineering(df):
  # Create a column to indicate funder with atleast 5 pumps maintained.
  value_count_funder = df.funder.value_counts()
  df['funder_aleast_5'] = df['funder'].apply(atleast, 
                                            args=(value_count_funder,))
  
  # Create a column to indicate installer with atleast 5 pumps maintained.
  value_count_installer = df.installer.value_counts()
  df['installer_aleast_5'] = df['installer'].apply(atleast, 
                                            args=(value_count_installer,))
  
  # Apply mean for missing values of latitude and longitude
  mean_longitude = df['longitude'].mean()
  df['longitude'] = df['longitude'].apply(lambda x: mean_longitude if round(x, 2) == 0 else x)
  mean_latitude = df['latitude'].mean()
  df['latitude'] = df['latitude'].apply(lambda x: mean_latitude if round(x, 2) == 0 else x)
  
  # Grouping wpt_name, subvillage based on 1st alphabet
  df['wpt_name_character_grouping'] = df['wpt_name'].apply(character_grouping)
  df['subvillage_character_grouping'] = df['subvillage'].apply(character_grouping)
  
  # Classify lga based on Rural, Urban and others
  df['lga_engineered'] = df['lga'].apply(classify_lga)
  
  # Grouping ward, scheme_name based on 1st alphabet
  df['ward_character_grouping'] = df['ward'].apply(character_grouping)
  df['scheme_name_character_grouping'] = df['scheme_name'].apply(character_grouping)
  
  """
  # Grouping based on prefix
  df['funder_prefix_grouping'] = df['funder'].apply(prefix_grouping)
  df['installer_prefix_grouping'] = df['installer'].apply(prefix_grouping)
  df['wpt_name_prefix_grouping'] = df['wpt_name'].apply(prefix_grouping)
  df['subvillage_prefix_grouping'] = df['subvillage'].apply(prefix_grouping)
  df['lga_prefix_grouping'] = df['lga'].apply(prefix_grouping)
  df['ward_prefix_grouping'] = df['ward'].apply(prefix_grouping)
  df['scheme_name_prefix_grouping'] = df['scheme_name'].apply(prefix_grouping)
  """
  
  # Compute missing construction year
  ward_construction_year_dict = map_ward_construction_year(df)
  top_ward = df['ward'].describe().top
  top_ward_construction_year =  \
    int(df[df['ward'] == top_ward]['construction_year'].median())
  
  df['construction_year'] = df.apply(compute_construction_year, axis=1, 
                                     args=(ward_construction_year_dict,
                                          top_ward_construction_year,))
  
  # Compute age of well
  df['age'] = df.apply(compute_age, axis=1)
  
  # Fetch Year and Month of date recorded
  df['year_recorded'] = df['date_recorded'].apply(compute_year_recorded)
  df['month_recorded'] = df['date_recorded'].apply(compute_month_recorded)

feature_engineering(train_features_df)

In [0]:
# Selecting independent and dependent variables.

X = train_features_df.drop(columns=['id', 'funder', 'installer', 'wpt_name', 
                                    'subvillage', 'lga','ward','scheme_name'])
y = train_labels_df.status_group

In [9]:
pd.set_option('display.max_columns', None)
train_features_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,funder_aleast_5,installer_aleast_5,wpt_name_character_grouping,subvillage_character_grouping,lga_engineered,ward_character_grouping,scheme_name_character_grouping,age,year_recorded,month_recorded
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,NaN,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1.0,1.0,NaN,m,rural,m,r,12,2011,3
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1.0,1.0,z,n,rural,n,NaN,3,2013,3
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1.0,1.0,k,m,rural,n,n,4,2013,2
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1.0,1.0,z,m,rural,n,NaN,27,2013,1
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,2004,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,0.0,1.0,s,k,rural,n,NaN,7,2011,7


In [0]:
# Split data into train and test using k-fold cross-validation
# with independent test data set.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=42
                                                   )

In [11]:
# Get quick initial metrics estimate.

# Using sklearn accuracy_score
import numpy as np
from sklearn.metrics import accuracy_score

majority_class = y_train.mode()[0]
prediction = np.full(shape=y_train.shape, 
                     fill_value=majority_class)

print(f'accuracy score {accuracy_score(y_train, prediction)}')


# Using simple pandas value counts method
print(y_train.value_counts(normalize=True))

accuracy score 0.542334455667789
functional                 0.542334
non functional             0.384871
functional needs repair    0.072795
Name: status_group, dtype: float64


In [0]:
# Data pre-processing, Feature selection and Model selection.

# Imports for pipeline
from sklearn.pipeline import make_pipeline

import category_encoders as ce
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [0]:
# Create pipeline
pipeline = make_pipeline(\
                         ce.BinaryEncoder(),
                         RobustScaler(),
                         XGBClassifier(learning_rate=0.1, n_estimators=1000,
                                       max_depth=4, min_child_weight=6,
                                       gamma=0, subsample=0.8,
                                       colsample_bytree=0.8,
                                       objective= 'multi:softmax', num_class=3,
                                       scale_pos_weight=1,
                                       seed=42, n_jobs=4))

In [0]:
# Model validation. 

param_grid = {
    'xgbclassifier__max_depth': range(3, 10, 2),
    'xgbclassifier__min_child_weight': range(1, 6, 2)
}

gridsearch1 = GridSearchCV(pipeline, param_grid=param_grid, cv=2, 
                         scoring='accuracy', verbose=20)

gridsearch1.fit(X_train, y_train)

In [0]:
# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch1.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch1.best_params_)

**Output from 1st Iteration of parameter tuning.**

Cross Validation Score: 0.7924354657687991

Best Parameters: {'xgbclassifier__max_depth': 5, 'xgbclassifier__min_child_weight': 1}

In [0]:
# Iteration 2 of Hyper-parameter tuning reconfirming the boundary
param_grid2 = {
    'xgbclassifier__max_depth': [4, 5, 6],
    'xgbclassifier__min_child_weight': [1, 2]
}

gridsearch2 = GridSearchCV(pipeline, param_grid=param_grid2, cv=2, 
                         scoring='accuracy', verbose=20)

gridsearch2.fit(X_train, y_train)

In [0]:
# Interpret the results of Iteration 2

# Best cross validation score
print('Cross Validation Score:', gridsearch2.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch2.best_params_)

**Output from 2nd Iteration of parameter tuning.**

Cross Validation Score: 0.793625140291807

Best Parameters: {'xgbclassifier__max_depth': 6, 'xgbclassifier__min_child_weight': 1}

In [0]:
# Iteration 3 of Hyper-parameter tuning reconfirming max_depth
param_grid3 = {
    'xgbclassifier__max_depth': [5, 6, 7],
    'xgbclassifier__min_child_weight': [1]
}

gridsearch3 = GridSearchCV(pipeline, param_grid=param_grid3, cv=2, 
                         scoring='accuracy', verbose=20)

gridsearch3.fit(X_train, y_train)

In [0]:
# Interpret the results of Iteration 3

# Best cross validation score
print('Cross Validation Score:', gridsearch3.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch3.best_params_)

**Output from 3rd Iteration of parameter tuning.**

Cross Validation Score: 0.793625140291807

Best Parameters: {'xgbclassifier__max_depth': 6, 'xgbclassifier__min_child_weight': 1}

In [0]:
# Iteration 4 of Hyper-parameter tuning gamma

param_grid4 = {
    'xgbclassifier__max_depth': [6],
    'xgbclassifier__min_child_weight': [1],
    'xgbclassifier__gamma': [i/10.0 for i in range(0,5)]
}

gridsearch4 = GridSearchCV(pipeline, param_grid=param_grid4, cv=2, 
                         scoring='accuracy', verbose=20)

gridsearch4.fit(X_train, y_train)

In [0]:
# Interpret the results of Iteration 4

# Best cross validation score
print('Cross Validation Score:', gridsearch4.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch4.best_params_)

**Output from 4th Iteration of parameter tuning.**

Cross Validation Score: 0.793625140291807

Best Parameters: {'xgbclassifier__gamma': 0.0, 'xgbclassifier__max_depth': 6, 'xgbclassifier__min_child_weight': 1}

In [0]:
# Iteration 5 of Hyper-parameter tuning learning_rate

param_grid5 = {
    'xgbclassifier__max_depth': [6],
    'xgbclassifier__min_child_weight': [1],
    'xgbclassifier__gamma': [0.0],
    'xgbclassifier__learning_rate': [0.1, 0.2, 0.3, 0.5]
}

gridsearch5 = GridSearchCV(pipeline, param_grid=param_grid5, cv=2, 
                         scoring='accuracy', verbose=20)

gridsearch5.fit(X_train, y_train)

In [0]:
# Interpret the results of Iteration 5

# Best cross validation score
print('Cross Validation Score:', gridsearch5.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch5.best_params_)

**Output from 5th Iteration of parameter tuning.**

Cross Validation Score: 0.793625140291807

Best Parameters: {'xgbclassifier__gamma': 0.0, 'xgbclassifier__learning_rate': 0.1, 'xgbclassifier__max_depth': 6, 'xgbclassifier__min_child_weight': 1}

In [22]:
# Iteration 6 of Hyper-parameter tuning learning_rate with values less than 0.1

param_grid6 = {
    'xgbclassifier__max_depth': [6],
    'xgbclassifier__min_child_weight': [1],
    'xgbclassifier__gamma': [0.0],
    'xgbclassifier__learning_rate': [0.0001, 0.001, 0.01, 0.1]
}

gridsearch6 = GridSearchCV(pipeline, param_grid=param_grid6, cv=2, 
                         scoring='accuracy', verbose=20)

gridsearch6.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] xgbclassifier__gamma=0.0, xgbclassifier__learning_rate=0.0001, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  xgbclassifier__gamma=0.0, xgbclassifier__learning_rate=0.0001, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=1, score=0.7345124797988867, total= 6.1min
[CV] xgbclassifier__gamma=0.0, xgbclassifier__learning_rate=0.0001, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.2min remaining:    0.0s


[CV]  xgbclassifier__gamma=0.0, xgbclassifier__learning_rate=0.0001, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=1, score=0.7320642902038251, total= 6.1min
[CV] xgbclassifier__gamma=0.0, xgbclassifier__learning_rate=0.001, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=1 


...

[CV]  xgbclassifier__gamma=0.0, xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=1, score=0.7935446220147244, total= 6.0min
[CV] xgbclassifier__gamma=0.0, xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=1 


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed: 43.2min remaining:    0.0s


[CV]  xgbclassifier__gamma=0.0, xgbclassifier__learning_rate=0.1, xgbclassifier__max_depth=6, xgbclassifier__min_child_weight=1, score=0.7937056657986891, total= 6.0min


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 49.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 49.3min finished


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('binaryencoder', BinaryEncoder(cols=None, drop_invariant=False, handle_unknown='impute',
       impute_missing=True, return_df=True, verbose=0)), ('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('xgbclassifier', XGBClassi...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=42, silent=True, subsample=0.8))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'xgbclassifier__max_depth': [6], 'xgbclassifier__min_child_weight': [1], 'xgbclassifier__gamma': [0.0], 'xgbclassifier__learning_rate': [0.0001, 0.001, 0.01, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=20)

In [23]:
# Interpret the results of Iteration 6

# Best cross validation score
print('Cross Validation Score:', gridsearch6.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch6.best_params_)

Cross Validation Score: 0.793625140291807
Best Parameters: {'xgbclassifier__gamma': 0.0, 'xgbclassifier__learning_rate': 0.1, 'xgbclassifier__max_depth': 6, 'xgbclassifier__min_child_weight': 1}


**Output from 6th Iteration of parameter tuning.**

Cross Validation Score: 0.793625140291807

Best Parameters: {'xgbclassifier__gamma': 0.0, 'xgbclassifier__learning_rate': 0.1, 'xgbclassifier__max_depth': 6, 'xgbclassifier__min_child_weight': 1}

In [26]:
#Get the best model and check it against test data set.

# Predict with X_test features
y_pred = gridsearch6.predict(X_test)

# Compare predictions to y_test labels
test_score = accuracy_score(y_test, y_pred)
print('Accuracy Score on test data set:', test_score)

Accuracy Score on test data set: 0.8066666666666666


In [0]:
test_features_df = pd.read_csv('test_features.csv', na_values=nan_values_list)
feature_engineering(test_features_df)

X_submission = test_features_df.drop(columns =['id', 'funder', 'installer', 'wpt_name', 'subvillage', 
                                    'lga','ward','scheme_name'])

# Predict with X_submission features
y_submission = gridsearch5.predict(X_submission)

y_submission_df = pd.DataFrame(y_submission, columns=['status_group'])
output_for_submission = test_features_df.join(y_submission_df).loc[:, ['id','status_group']]

In [19]:
output_for_submission.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [28]:
print(output_for_submission.status_group.value_counts())
print(output_for_submission.shape)

functional                 8614
non functional             5132
functional needs repair     612
Name: status_group, dtype: int64
(14358, 2)


In [0]:
print(output_for_submission.to_csv(index=False))